In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import math

from pandas import DataFrame, Series
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [2]:
def normalization(columns): # -1~1사이의 값으로 정규화
    MinMax = (columns - columns.min(axis=0)) / (columns.max(axis=0) - columns.min(axis=0))
    MinMax = (MinMax * 2) - 1
    MinMax
    return MinMax

In [3]:
def print_graph(MinMax, num): # num is cut range
    length = len(MinMax)
    plt.figure(figsize=(20, 5)) # graph size
    plt.xticks(np.arange(0, length+1, num))
    plt.plot(MinMax)
    plt.show()

In [4]:
def cut_count(MinMax, num, cnt): # 왼발, 오른발의 데이터를 잘라서 구분
    count = int((len(MinMax)-num) / cnt)
    step = []
    temp = 0
    for i in range(0, count):
        temp = num+i*cnt
        step.append(MinMax[temp:temp+cnt])
    return step

In [5]:
def seperate_z(x) :
    if (x < 0.6):
        return False
    else : return True

In [6]:
def LPF(rawData):
    rawData['targetData'] = rawData['AccY']

    ts = 0.01 # 100Hz
    tau = 0.1
#     print_graph(rawData['targetData'], 50)
#     print(rawData['targetData'])
    tmpResult = Series(np.zeros(len(rawData.targetData)))
    tmpResult[0] = rawData.targetData[0]

    for i in np.arange(1, len(rawData.targetData)):
        tmpResult[i] = (tau*tmpResult[i-1] + ts * rawData.targetData[i]) / (ts + tau)

    rawData['LPF'] = tmpResult

#     plt.plot(rawData.SampleRate, rawData.targetData, '#afafaf',  rawData.SampleRate, rawData.LPF_X, 'b')
#     plt.grid(True)
#     plt.show()
#     print(rawData)
    return rawData

In [7]:
def ACF (step, cnt):
    left = []
    right = []
    acf = []
    for i in range(1,len(step)) :
        left = step[i-1]
        right = step[i]
#         print(left, right)
        acf.append(np.corrcoef(left, right))
#         print(acf)
    return acf

In [8]:
def compare (acf) :
    answer = 0
    for i in acf:
        answer += abs(i[0][1])
    answer = answer / len(acf)
    return answer

In [64]:
def main(rawData, cnt):
    LPFData = LPF(rawData)
#     print(LPFData)
    MinMax = normalization(LPFData['LPF'])
#     print_graph(MinMax, 80)
#     print(len(MinMax))
    step = cut_count(MinMax, 0, cnt)
#     print(len(step))
    acf = ACF(step, cnt)
    result = compare(acf)
#     print(result)
    if (result < 0.4) :
#         print('\033[34m', True)
        return True
    else : 
#         print('\033[31m', False)
        return False
#     if (result == True):
#         print('\033[34m', result)
#     else : print('\033[31m', result)
#     return result

In [31]:
rawData = pd.read_csv('0529Data/0529_user1/NormalGait1.csv')
column = rawData[['AccX', 'AccY', 'AccZ']]
rawData_0 = pd.read_csv('0529Data/0529_user1/AbnormalGait1.csv')
column_0 = rawData_0[['AccX', 'AccY', 'AccZ']]
rawData_1 = pd.read_csv('0529Data/abnormal/abnormalWalk1.csv')
column_1 = rawData_1[['AccX', 'AccY', 'AccZ']]
main(rawData, 139)
main(rawData_0, 139)
main(rawData_1, 139)

0.3610528734091765
0.5887180240713319
0.10060054340243313


In [59]:
for i in range(0, 10):
    rawData = pd.read_csv(f'0529Data/abnormal/abnormalWalk{i}.csv')
    result = main(rawData, 139)
    if (result < 0.4) :
         print('\033[34m', True)
    else : print('\033[31m', False)

 True
 True
 True
 True
 True
 True
 True
 True
 True
 True


In [102]:
f = open("result.csv", "w", encoding='utf-8', newline='')
wr = csv.writer(f)
for i in range(1, 201):
    cnt = 0
    for j in range(1, 6):
        rawData = pd.read_csv(f'0529Data/0529_user10/AbnormalGait{j}.csv')
        result = main(rawData, i)
#         print(result)
        if (result == False):
            cnt += 1
    wr.writerow([i,cnt*20])
    print("--------",i, " : ", cnt*20, "% ----------")
f.close()

-------- 1  :  100 % ----------
-------- 2  :  100 % ----------
-------- 3  :  100 % ----------
-------- 4  :  100 % ----------
-------- 5  :  100 % ----------
-------- 6  :  100 % ----------
-------- 7  :  100 % ----------
-------- 8  :  100 % ----------
-------- 9  :  100 % ----------
-------- 10  :  100 % ----------
-------- 11  :  100 % ----------
-------- 12  :  100 % ----------
-------- 13  :  100 % ----------
-------- 14  :  100 % ----------
-------- 15  :  100 % ----------
-------- 16  :  100 % ----------
-------- 17  :  100 % ----------
-------- 18  :  100 % ----------
-------- 19  :  100 % ----------
-------- 20  :  100 % ----------
-------- 21  :  100 % ----------
-------- 22  :  100 % ----------
-------- 23  :  100 % ----------
-------- 24  :  100 % ----------
-------- 25  :  100 % ----------
-------- 26  :  100 % ----------
-------- 27  :  100 % ----------
-------- 28  :  100 % ----------
-------- 29  :  100 % ----------
-------- 30  :  100 % ----------
-------- 31  :  100

In [52]:
f = open("result.csv", "w", encoding='utf-8', newline='')
wr = csv.writer(f)
# for j in range(1, 5):
# #     wr.writerow([f'user{j}'])
#     for i in range(0, 10):
#         rawData = pd.read_csv(f'0529Data/user{j}/normalWalk{i}.csv')
#         result = main(rawData, 139)
#         wr.writerow([result])
# wr.writerow(['user9'])
for i in range(0, 10):
    rawData = pd.read_csv(f'0529Data/abnormal/abnormalWalk{i}.csv')
    result = main(rawData, 139)
    wr.writerow([result])
    
# wr.writerow(['user10'])
# for i in range(1, 6):
#     rawData = pd.read_csv(f'0529Data/0529_user{j}/AbnormalGait{i}.csv')
#     result = main(rawData, 139)
#     wr.writerow([result])
f.close()

800
0.12796183635498204
800
0.10060054340243313
800
0.26671338012257967
801
0.21027200393565593
801
0.21245933087813762
801
0.06110510932784961
800
0.20364968261843797
800
0.05778750766842623
801
0.1864989383590704
800
0.1266190430846457
